# Laboratorium 3

In [1]:
import numpy as np

np.random.seed(123)

In [28]:
class CRR:
    def __init__(self, S0, d, u, r, T):
        if not (d < 1 + r < u):
            raise ValueError('in CRR model it must be d < 1 + r < u')
        self.S0 = S0
        self.d = d
        self.u = u
        self.r = r
        self.T = T
        self.p = (1 + r - d)/(u - d)
        
    def get_S_trajectory(self, trajectory):
        return np.cumprod(trajectory) * self.S0
    
    def get_rational_price(self, option, trajectory=None):
            if trajectory is None:
                C0 = self.p * self.get_rational_price(option, np.array([self.u])) + (1 - self.p) * self.get_rational_price(option, np.array([self.d]))
                C0 /= 1 + self.r
                return C0
            elif len(trajectory) != self.T:
                tu = np.append(np.ndarray.copy(trajectory), self.u)
                td = np.append(np.ndarray.copy(trajectory), self.d)
                Ct = self.p * self.get_rational_price(option, tu) + (1 - self.p) * self.get_rational_price(option, td)
                Ct /= 1 + self.r
                return Ct
            else:
                S_trajectory = self.get_S_trajectory(trajectory)
                return option.get_final_price(S_trajectory)

In [13]:
class Option:
    def __init__(self, K):
        self.K = K
    
    def get_final_price(self, S_trajectory):
        pass

In [17]:
class CallOption(Option):
    def __init__(self, K):
        super().__init__(K)
        
    def get_final_price(self, S_trajectory):
        return max(S_trajectory[-1] - self.K, 0.0)

In [16]:
class PutOption(Option):
    def __init__(self, K):
        super().__init__(K)
        
    def get_final_price(self, S_trajectory):
        return max(self.K - S_trajectory[-1], 0.0)

In [32]:
class SomeOtherOption(Option):
    def __init__(self, K=0.0):
        super().__init__(K)
        
    def get_final_price(self, S_trajectory):
        return np.max(S_trajectory)

In [29]:
crr = CRR(100, 0.8, 1.3, 0.1, 10)

In [20]:
call = PutOption(90)

In [30]:
crr.get_rational_price(call)

1.6689609138656192

In [24]:
put = PutOption(90)

In [31]:
crr.get_rational_price(put)

1.6689609138656192

In [33]:
soo = SomeOtherOption()
crr.get_rational_price(soo)

116.59159369509308